<a href="https://colab.research.google.com/github/michaelwaheb/michaelwaheb/blob/main/GPC_Predication_AccuracyTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import requests
import time
import os

EXCEL_FILENAME = "/content/sample_data/GPC_Test_Products.xlsx"
OUTPUT_FILENAME = "classification_detailed_results.xlsx"
API_URL = "https://ml.gsegdc.com:2053/v1/classify"

def main():
    # Check if Excel file exists
    if not os.path.exists(EXCEL_FILENAME):
        print(f"❌ Excel file '{EXCEL_FILENAME}' not found in current directory.")
        return

    # Ask user for API key
    api_key = input("🔑 Enter your GPC API Key: ").strip()
    if not api_key:
        print("❌ API Key is required.")
        return

    # Load Excel file
    df = pd.read_excel(EXCEL_FILENAME)

    print(f"📂 Found '{EXCEL_FILENAME}' with {len(df)} products.\n\n Starting Measure Accuracy for each product description...\n")

    results = []
    total_accuracy = 0

    for index, row in df.iterrows():
        product_description = row["Product Name"]
        expected_bricks = set(str(row["BrickCode"]).replace(" ", "").split(','))

        payload = {
            "gtin": "62211234567898",
            "product_description": product_description
        }

        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
            "X-API-Key": api_key
        }

        try:
            response = requests.post(API_URL, json=payload, headers=headers)
            response.raise_for_status()
            data = response.json()

            predicted_bricks = [item["brick_code"] for item in data.get("result", {}).get("matched_bricks", [])]
            predicted_set = set(predicted_bricks)
            correct_matches = expected_bricks.intersection(predicted_set)

            precision = len(correct_matches) / len(predicted_bricks) if predicted_bricks else 0
            recall = len(correct_matches) / len(expected_bricks) if expected_bricks else 0

            row_accuracy = 2 * (precision * recall) / (precision + recall) * 100 if precision + recall else 0
            total_accuracy += row_accuracy

            print(f"[{index + 1}/{len(df)}] '{product_description[:40]}...' → "
                  f"Accuracy: {row_accuracy:.2f}% \n| ExpectedBricksCode: {expected_bricks} | PredictedBricksCode: {predicted_bricks}")

            results.append({
                "Product Name": product_description,
                "Expected Bricks": ', '.join(expected_bricks),
                "Predicted Bricks": ', '.join(predicted_bricks),
                "Matched Count": len(correct_matches),
                "Precision (%)": round(precision * 100, 2),
                "Recall (%)": round(recall * 100, 2),
                "Row Accuracy (F1 %)": round(row_accuracy, 2)
            })

            time.sleep(0.5)  # Respect API rate limit

        except Exception as e:
            print(f"❌ Error processing row {index + 1}: {e}")
            results.append({
                "Product Name": product_description,
                "Expected Bricks": ', '.join(expected_bricks),
                "Predicted Bricks": "API Error",
                "Matched Count": 0,
                "Precision (%)": 0.0,
                "Recall (%)": 0.0,
                "Row Accuracy (F1 %)": 0.0
            })


    overall_accuracy = total_accuracy / len(df)
    print(f"\n✅ Accuracy measurement complete and measuer Overall Average Accuracy")
    print(f"🎯 Overall Average Accuracy (F1-based): {overall_accuracy:.2f}%\n")

if __name__ == "__main__":
    main()



🔑 Enter your GPC API Key: 26269a82-b4f8-43c4-93f8-68af329f2443
📂 Found '/content/sample_data/GPC_Test_Products.xlsx' with 40 products.

 Starting Measure Accuracy for each product description...

[1/40] 'Oven 36L Mechanical Full Option Black 36...' → Accuracy: 80.00% 
| ExpectedBricksCode: {'10001950', '10001968', '10001974'} | PredictedBricksCode: ['10001950', '10001974']
[2/40] 'Front-Loading, 8Kg, Dark Silver, 1200Rpm...' → Accuracy: 0.00% 
| ExpectedBricksCode: {'10001959'} | PredictedBricksCode: ['10006374']
[3/40] '3-Taps Water Dispenser, Without Cabinet...' → Accuracy: 50.00% 
| ExpectedBricksCode: {'10003710', '10003712'} | PredictedBricksCode: ['10003712', '10003713']
[4/40] 'Feta Cheese Tetra Pack 250 G...' → Accuracy: 80.00% 
| ExpectedBricksCode: {'10000028', '10000029'} | PredictedBricksCode: ['10000028', '10000029', '10000030']
[5/40] 'Double Cream White Soft Cheese...' → Accuracy: 100.00% 
| ExpectedBricksCode: {'10000028', '10000029'} | PredictedBricksCode: ['10000028',

In [7]:
import pandas as pd
import requests
import time

# Load the Excel file
df = pd.read_excel("/content/sample_data/GPC_Test_Products.xlsx")

# API configuration
API_URL = "https://ml.gsegdc.com:2053/v1/classify"
API_KEY = "26269a82-b4f8-43c4-93f8-68af329f2443"  # <<< Paste your API key here

# Store row-level accuracy
results = []
total_accuracy = 0

for index, row in df.iterrows():
    product_description = row["Product Name"]

    # Convert comma-separated BrickCode string into a set
    expected_bricks = set(str(row["BrickCode"]).replace(" ", "").split(','))

    payload = {
        "gtin": "62211234567898",
        "product_description": product_description
    }

    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "X-API-Key": API_KEY
    }

    try:
        response = requests.post(API_URL, json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()

        predicted_bricks = [item["brick_code"] for item in data.get("result", {}).get("matched_bricks", [])]
        predicted_set = set(predicted_bricks)
        correct_matches = expected_bricks.intersection(predicted_set)

        precision = len(correct_matches) / len(predicted_bricks) if predicted_bricks else 0
        recall = len(correct_matches) / len(expected_bricks) if expected_bricks else 0

        if precision + recall == 0:
            row_accuracy = 0.0
        else:
            row_accuracy = 2 * (precision * recall) / (precision + recall) * 100

        total_accuracy += row_accuracy

        print(f"[{index + 1}] {product_description[:50]}... -> Row Accuracy: {row_accuracy:.2f}% "
              f"| Expected: {expected_bricks} | Predicted: {predicted_bricks}")

        results.append({
            "Product Name": product_description,
            "Expected Bricks": ', '.join(expected_bricks),
            "Predicted Bricks": ', '.join(predicted_bricks),
            "Matched Count": len(correct_matches),
            "Precision": round(precision * 100, 2),
            "Recall": round(recall * 100, 2),
            "Row Accuracy (F1 %)": round(row_accuracy, 2)
        })

        time.sleep(0.5)  # Respect API limits

    except Exception as e:
        print(f"❌ Error in row {index + 1}: {e}")
        results.append({
            "Product Name": product_description,
            "Expected Bricks": ', '.join(expected_bricks),
            "Predicted Bricks": "API Error",
            "Matched Count": 0,
            "Precision": 0.0,
            "Recall": 0.0,
            "Row Accuracy (F1 %)": 0.0
        })

# Save results
results_df = pd.DataFrame(results)
results_df.to_excel("classification_detailed_results.xlsx", index=False)

# Print overall average accuracy
overall_accuracy = total_accuracy / len(df)
print(f"\n✅ Overall Average Accuracy (F1-based): {overall_accuracy:.2f}%")


[1] Oven 36L Mechanical Full Option Black 3602 Rcl... -> Row Accuracy: 80.00% | Expected: {'10001950', '10001968', '10001974'} | Predicted: ['10001950', '10001974']
[2] Front-Loading, 8Kg, Dark Silver, 1200Rpm, Pc Door,... -> Row Accuracy: 0.00% | Expected: {'10001959'} | Predicted: ['10006374']
[3] 3-Taps Water Dispenser, Without Cabinet... -> Row Accuracy: 50.00% | Expected: {'10003710', '10003712'} | Predicted: ['10003712', '10003713']
[4] Feta Cheese Tetra Pack 250 G... -> Row Accuracy: 80.00% | Expected: {'10000028', '10000029'} | Predicted: ['10000028', '10000029', '10000030']
[5] Double Cream White Soft Cheese... -> Row Accuracy: 100.00% | Expected: {'10000028', '10000029'} | Predicted: ['10000028', '10000029']
[6] Fingers Hazelnut Wafer New Size Display Box Includ... -> Row Accuracy: 66.67% | Expected: {'10000161'} | Predicted: ['10000304', '10000161']
[7] Frozen Eggplant... -> Row Accuracy: 66.67% | Expected: {'10000005'} | Predicted: ['10000005', '10006125']
[8] Frozen Strawb